In [16]:
import os
import numpy as np
import pandas as pd
from scipy.optimize import nnls 

In [17]:
organ="AA_heart"
# organ="AA_muscle"
# organ="AA_liver"
# organ="AA_kidney"
path=f"C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/{organ}/"#proportion_pxt_neh/"  
Protiens=np.unique([x.replace('__','').replace('.csv','') for x in os.listdir(path) if ("_MOUSE.csv" in x)])
times=[1,2,3,6,7,9,13,16,21,24,31] 

In [18]:
# Protiens=["FAS_MOUSE","CPSM_MOUSE"]
time=31
pw=0.046

In [19]:
res=[]
for time in times[3:-1]:
    print(f"<============================= Time = {time} =============================>")
    for protien in Protiens:
        try:
            data=pd.read_csv(path+f"__{protien}.csv")
            data=data[data.exp_Time==time]        
#             data=data[data.new_RMSE.astype('float')<0.005]
            data=data[data.exp_Time==time].reset_index(drop="True")
        #         print(protien,data.shape)
        except:
            continue
            
        if(data.shape[0]==0):
            continue


        for pep_index in range(data.shape[0]):
            exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
            b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]]) 

            A=[]
            for i in range(6):
                A.append(([0]*i)+exp_0[:6-i])
            A=np.array(A).transpose()
            x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    #         x,residual=nnls(A,b)

            ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
            residual=sum([ abs(b[i]-y) for index,y in enumerate(ypred)])        

    #         print("\n",data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],"\n",np.corrcoef(x,exp_0)[1][0])

            neh= (x[2]/x[1])*2* ((1-pw)/pw)  + 1
            res.append([protien,data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh])
        
res=pd.DataFrame(res)
res.columns=["protien","Peptide","Charge","True NEH","e NEH",]
res["dif"]=abs(res["True NEH"]- res["e NEH"]) 
res=res.sort_values(by="dif")
print(res.shape)

<============================= Time = 6 =============================>
<============================= Time = 7 =============================>
<============================= Time = 9 =============================>
<============================= Time = 13 =============================>
<============================= Time = 16 =============================>
<============================= Time = 21 =============================>
<============================= Time = 24 =============================>
(1457, 6)


In [20]:
print(res.shape)
res=res[res.dif <1.25]
print(res.shape)
res=res.drop_duplicates(subset=["protien","Peptide"])
print(res.shape) 
# 
# res=res[(res["e NEH"]<40) & (res["e NEH"]>15)]
print(res.shape) 

res=res.sort_values(by="dif")
res=res.reset_index(drop=True)
res=res.iloc[:200,:]

(1457, 6)
(176, 6)
(125, 6)
(125, 6)


In [21]:
all_AAs=['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p','q', 'r', 's', 't', 'v', 'w', 'y']
def charcount(pepSeq):
    res={key: 0 for key in all_AAs}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [22]:
def sort_pep(peptidesinfo,peptides):
    temp={p: peptidesinfo[p]["NEH"] for i,p in enumerate(peptides)}
    temp=dict(sorted(temp.items(), key=lambda item: item[1])) 
    return list(temp.keys()) 

In [23]:

peptidesinfo=dict()
peptides=[]
for k in range(len(res)):
    peptides.append(res.loc[k,"Peptide"]+"_"+str(res.loc[k,"Charge"]))
#     peptidesinfo[peptides[-1]]={"T_NEH":float(res.loc[k,"True NEH"]),"Seq":res.loc[k,"Peptide"],"charcount":charcount(res.loc[k,"Peptide"])}
    peptidesinfo[peptides[-1]]={"NEH":float(res.loc[k,"e NEH"]),"Seq":res.loc[k,"Peptide"],"charcount":charcount(res.loc[k,"Peptide"])}

visited=set()
visited.add(peptides[0].split('_')[1])
coefficients=[]
peptides=sort_pep(peptidesinfo,peptides)
for k in range(len(peptides)):

    pep1=peptidesinfo[peptides[k]]  
    visited.add(peptides[k].split('_')[1])
 
    coefficients.append([pep1["charcount"][c] for c in all_AAs]+[pep1["NEH"]])
    
coefficients=np.array(coefficients)
sol=list(nnls(coefficients[:,:-1],coefficients[:,-1])[0])

res=pd.DataFrame()
res["AA"]=all_AAs
res["True_value"]=[4.0,1.62,1.89,3.95,0.32,2.06,2.88,1.0,0.6,0.54,1.12,1.89,2.59,3.95,3.43,2.61,0.2,0.56,0.08,0.42]
res["est_value"]=sol

## ci 95%
ypred=[np.dot(coefficients[index,:-1],sol) for index in range(coefficients.shape[0])]
sigma=sum([ (coefficients[index,-1]-x)**2 for index,x in enumerate(ypred)])/(len(ypred)-20-1)
cov=((sigma**2)*np.linalg.inv(np.matmul(coefficients[:,:-1].transpose(),coefficients[:,:-1])))
var=[cov[x][x] for x in range(20)] 
ci=[1.96*(var[i]**0.5) for i in range(20)]
res["CI"]=ci

res.sort_values(by="AA")

,AA,True_value,est_value,CI
0,a,4.00,4.050337,0.069066
1,c,1.62,1.734180,0.292564
2,d,1.89,2.122760,0.105295
3,e,3.95,3.941159,0.076924
4,f,0.32,0.290411,0.128596
5,g,2.06,1.901482,0.089748
6,h,2.88,2.881034,0.172196
7,i,1.00,0.984634,0.109510
8,k,0.60,0.181999,0.202075
9,l,0.54,0.615465,0.084733
